In [ ]:
import cv2
import numpy as np
import json

In [2]:
def interesting(path,dimensions):
    def typization(string):
        if string=='chart_title':
            return (1,1,1)
        elif string=='axis_title':
            return (2,2,2)
        elif string=='tick_label':
            return (3,3,3)
        elif string=='plot-bb':
            return (4,4,4)
        elif string=='axes':
            return (5,5,5)
        elif string=="bars":
            return (6,6,6)
        elif string=="boxplots":
            return (7,7,7)
        elif string=="dot points":
            return (8,8,8)
        elif string=="lines":
            return (9,9,9)
        elif string=="scatter points":
            return (10,10,10)
# creator = np.array(dimensions[1]*[dimensions[2]*[0]])
    creator = np.zeros((dimensions[0], dimensions[1]), dtype=np.uint8)
    creator = cv2.cvtColor(creator, cv2.COLOR_GRAY2BGR)
    print('img: ', dimensions,'\n')
    s=""
    with open(path) as f:
        s=f.read()
    gen=json.loads(s)
    gen2=gen['plot-bb']
    cv2.fillPoly(creator, [np.array([[(gen2['x0']+gen2['width'],gen2['y0'])],[(gen2['x0'],gen2['y0'])],[(gen2['x0'],gen2['y0']+gen2['height'])],[(gen2['x0']+gen2['width'],gen2['y0']+gen2['height'])]],dtype=np.int32)], typization('plot-bb'))
    for i in range(len(gen['text'])):
        if len(gen['text'][i]['polygon'])!=0:
            gen1=gen['text'][i]['polygon']
            cv2.fillPoly(creator, [np.array([[(gen1['x0'],gen1['y0'])],[(gen1['x1'],gen1['y1'])],[(gen1['x2'],gen1['y2'])],[(gen1['x3'],gen1['y3'])]],dtype=np.int32)], typization(gen['text'][i]['role']))
    axe=gen['axes']
    axex=axe['x-axis']['ticks']
    axey=axe['y-axis']['ticks']
    for i in range(len(axex)):
        cv2.fillPoly(creator, [np.array([[(axex[i]['tick_pt']['x'],axex[i]['tick_pt']['y'])],[(axex[i]['tick_pt']['x'],axex[i]['tick_pt']['y']-gen2['height'])]],dtype=np.int32)], typization('axes'))
    for i in range(len(axey)):
        cv2.fillPoly(creator, [np.array([[(axey[i]['tick_pt']['x'],axey[i]['tick_pt']['y'])],[(axey[i]['tick_pt']['x']+gen2['width'],axey[i]['tick_pt']['y'])]],dtype=np.int32)], typization('axes'))
    if len(gen["visual-elements"]["bars"])!=0:
        for el in gen["visual-elements"]["bars"]:
            cv2.fillPoly(creator, [np.array([[(el['x0']+el['width'],el['y0'])],[(el['x0'],el['y0'])],[(el['x0'],el['y0']+el['height'])],[(el['x0']+el['width'],el['y0']+el['height'])]],dtype=np.int32)], typization('bars'))
    elif len(gen["visual-elements"]["boxplots"])!=0:
        ...
    elif len(gen['visual-elements']['dot points'])!=0:
        for i in gen['visual-elements']['dot points'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 4, typization('dot points'), 8)
    elif len(gen['visual-elements']['lines'])!=0:
        k=[]
        for i in gen['visual-elements']['lines'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 2, typization('dot points'), 5)
        for i in range(1,len(gen['visual-elements']['lines'][0])):
            cv2.drawContours(creator,[np.array([(int(gen['visual-elements']['lines'][0][i-1]['x']),int(gen['visual-elements']['lines'][0][i-1]['y'])),(int(gen['visual-elements']['lines'][0][i]['x']),int(gen['visual-elements']['lines'][0][i]['y']))])],-1,typization('lines'),3)
    elif len(gen['visual-elements']['scatter points'])!=0:
        for i in gen['visual-elements']['scatter points'][0]:
            cv2.circle(creator, (int(i['x']), int(i['y'])), 2, typization('dot points'), 5)
# cv2.imwrite("mask.jpg",creator)
    return creator

In [3]:
def reconstruct(img: np.ndarray, mask: np.ndarray, tup: tuple):
    if tup == (1, 1, 1): #Chart title
        coords = np.where(mask[:,:,] == tup) #Ищем координаты полигона

        y_begin, y_end = coords[0].min(), coords[0].max() #Выделяем полигон
        x_begin, x_end = coords[1].min(), coords[1].max()

        return img[y_begin:y_end, x_begin:x_end] #Возвращаем обрезанное изображение
    
    if tup == (2, 2, 2): #Axis title
        coords = np.where(mask[:,:,] == tup) #Ищем координаты полигона
        fst_border = [] #Границы y координат
        snd_border = [] #Границы х координат

        for i in range(len(coords[0])):
            if coords[0][i] - coords[0][i + 1] < -1: #Все координаты идут в порядке возрастания, потому мы ищем резкий скачок значения в списке
                fst_border.append((coords[0][0], coords[0][i]))
                snd_border.append((coords[0][i + 1], coords[0].max()))
                break
        
        for i in range(len(coords[1])):
            if coords[1][i] - coords[1][i + 1] < -1:
                fst_border.append((coords[1][0], coords[1][i]))
                snd_border.append((coords[1][i + 1], coords[1].max()))
                break
        
        img1 = img[fst_border[0][0]:fst_border[0][1],fst_border[1][0]:fst_border[1][1]]
        img2 = img[snd_border[0][0]:snd_border[0][1],snd_border[1][0]:snd_border[1][1]]

        return img1, img2
    
    if tup == (3, 3, 3):
        coords = np.where(mask[:,:,] == tup) #Ищем координаты полигона
        y_tick_borders = []

        end_for_y_ticks = 0
        for i in range(len(coords[1])):
            if coords[1][i] - coords[1][i + 1] > 0:
                end_for_y_ticks = coords[1][i]
                break

        upper_border_y = coords[0][0]
        for i in range(len(coords[0])):
            try:
                if coords[0][i] - coords[0][i + 1] < -1:
                    y_tick_borders.append([(upper_border_y, coords[0][i]), (coords[1][0], end_for_y_ticks)])
                    upper_border_y = coords[0][i + 1]
            except BaseException:
                pass

        x_tick_border = img[upper_border_y:coords[0].max(), end_for_y_ticks:coords[1].max(),]

        return y_tick_borders, x_tick_border
    
    if tup == (4, 4, 4) or tup == (5, 5, 5):
        coords = np.where(mask[:,:,] == tup)

        y_begin, y_end = coords[0].min(), coords[0].max()
        x_begin, x_end = coords[1].min(), coords[1].max()

        return img[y_begin:y_end, x_begin:x_end + 1]
    
    else:
        coords = np.where(mask[:,:,] == tup)
        lower_border = np.where(mask[:,:,] == (3, 3, 3))[0].max()
        upper_border = coords[0].min()
        borders = []

        all_polygons = np.unique(coords[1])
        
        left_border = all_polygons[0]
        for i in range(len(all_polygons)):
            try:
                if all_polygons[i] - all_polygons[i + 1] < -1:
                    borders.append([(upper_border, lower_border), (left_border, all_polygons[i])])
                    left_border = all_polygons[i + 1]
            except BaseException:
                pass
        
        return borders

In [4]:
def script(img: np.ndarray, mask: np.ndarray):
    typ = {'chart_title': (1,1,1),
    'axis_title': (2,2,2),
    'tick_label': (3,3,3),
    'plot-bb': (4,4,4),
    'axes': (5,5,5),
    "bars": (6,6,6),
    "boxplots": (7,7,7),
    "dot points": (8,8,8),
    "lines": (9,9,9),
    "scatter points": (10,10,10)} #Цвет каждого полигона

    #Основной скрипт для восстановки полигонов
    ct = reconstruct(img, mask, typ["chart_title"])

    at1, at2 = reconstruct(img, mask, typ["axis_title"])
#    cv2.imwrite('./results/axis_title_y.jpg', at1)
#    cv2.imwrite('./results/axis_title_x.jpg', at2)

    tl_y, tl_x = reconstruct(img, mask, typ['tick_label'])
    y_tick_fragments = []
    for i in range(len(tl_y)):
        up, down = tl_y[i][0][0], tl_y[i][0][1]
        left, right = tl_y[i][1][0], tl_y[i][1][1]

        fragment = img[up:down, left:right,]

        y_tick_fragments.append(fragment)
    x_tick_fragments = tl_x

    pb = reconstruct(img, mask, typ["plot-bb"])

    axes = reconstruct(img, mask, typ["axes"])

    try:
        bars = reconstruct(img, mask, typ['bars'])
        bar_fragments = []
        for i in range(len(bars)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]
            bar_fragments.append(fragment)
        
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, bar_fragments
    except BaseException:
        pass
    try:
        bp = reconstruct(img, mask, typ["boxplots"])
        boxplot_fragments = []
        for i in range(len(bp)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]
            boxplot_fragments.append(fragment)
        
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, boxplot_fragments
    except BaseException:
        pass
    try:
        dp = reconstruct(img, mask, typ["dot points"])
        dp_fragments = []
        for i in range(len(dp)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]
            
            dp_fragments.append(fragment)
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, dp_fragments
    except BaseException:
        pass
    try:
        lines = reconstruct(img, mask, typ["lines"])
        line_fragments = []
        for i in range(len(lines)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]

            line_fragments.append(fragment)
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, line_fragments
    except BaseException:
        pass
    try:
        sp = reconstruct(mask, typ['scatter points'])
        scatter_fragments = []
        for i in range(len(sp)):
            up, down = bars[i][0][0], bars[i][0][1]
            left, right = bars[i][1][0] - 30, bars[i][1][1]

            fragment = img[up:down,left:right,]
            scatter_fragments.append(fragment)
        return ct, (at1, at2), (y_tick_fragments, x_tick_fragments), pb, axes, scatter_fragments
    except BaseException:
        pass